In [4]:
from keras.layers import *
from keras.models import *
import keras.backend as K
from keras.utils import plot_model
# encode time steps：Tx
# decode time steps: Ty
Tx = 5
Ty = 2


def getmodel(Tx, Ty, h, w, s):
    """
    X is a 4-D tensor, need to expand_dim outside(None, TimeSteps, h, w, c)
    pred is a 3-D tensor (None, h, w)
    :param Tx: input time steps
    :param Ty: output time steps
    :param h: input sample height:  longitude span
    :param w: input sample width:   latitude span
    s: decoder hidden shape
    :return: model
    """

    # encoding
    input = Input(shape=(Tx, h, w, 1), dtype='float')
    X = ConvLSTM2D(filters=48, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu', return_sequences=True, name='encode_layer_1')(input)

    e = ConvLSTM2D(filters=48, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu', return_sequences=False,  name='encode_layer_2')(X)
    # e.shape should be (batch, h, w, 48)

    e = Conv2D(1, (3, 3), padding='same')(e)

    

    # decoding
    # initial outputs
    outputs = []  # len = Ty, element shape = [h, w, 48]
    reshapor1 = Reshape((1, h * w))
    reshapor2 = Reshape((h, w))
    decode1 = LSTM(s, return_state=True)
    decode2 = LSTM(s, return_state=True)

    
    h1_in = Input(shape=(s, ))
    c1_in = Input(shape=(s, ))
    h2_in = Input(shape=(s, ))
    c2_in = Input(shape=(s, ))
    h1 = h1_in
    h2 = h2_in
    c1 = c1_in
    c2 = c2_in


    a1 = reshapor1(e)
    print(a1.get_shape())
    for i in range(Ty):
        h1, _, c1 = decode1(a1)
        print(h1.get_shape())
        a2 = reshapor1(h1)
        print(a2.get_shape())
        h2, _, c2 = decode2(a2)
        a1 = reshapor1(h2)

        out = reshapor2(h2)
        print(out.get_shape())
        outputs.append(out)

    pred = Add()(outputs)
    model = Model([input, h1_in, c1_in, h2_in, c2_in], pred)

    return model


model = getmodel(Tx, Ty, 40, 100, 4000)
print(model.summary(line_length=200))

Using TensorFlow backend.


(None, 1, 4000)
(None, 4000)
(None, 1, 4000)
(None, 40, 100)
(None, 4000)
(None, 1, 4000)
(None, 40, 100)
Model: "model_1"
________________________________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                                      Output Shape                                Param #                 Connected to                                                      
input_1 (InputLayer)                                              (None, 5, 40, 100, 1)                       0                                                                                         
________________________________________________________________________________________________________________________________________________________________________________________________________
encode_layer_1 (ConvLSTM2D)                                       (None, 

In [5]:
import numpy as np
data = np.load('nino3.4.npy')
print(data.shape)
print(data[1])

t, h, w = data.shape

Tx = 50

train_x = []
train_y = []
for i in range(t-Tx-1):
    train_x.append(data[i:i+Tx])
    train_y.append(data[i+Tx])

train_x = np.array(train_x)
train_y = np.array(train_y)

print(train_x.shape)
print(train_y.shape)

np.save('train_x.npy', train_x)
np.save('train_y.npy', train_y)

FileNotFoundError: ignored

In [0]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the fourth GPU
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

import numpy as np
data = np.load('nino3.4.npy')
print(data.shape)
print(data[1])

t, h, w = data.shape

Tx = 5

train_x = []
train_y = []
samples = 1000
for i in range(samples):
    train_x.append(data[i:i+Tx])
    train_y.append(data[i+Tx])

train_x = np.array(train_x)
train_y = np.array(train_y)

print(train_x.shape)
print(train_y.shape)


from keras.layers import *
from keras.models import *
import keras.backend as K
from keras.utils import plot_model
# encode time steps：Tx
# decode time steps: Ty


def getModel(Tx, h, w):
    """

    :param Tx:
    :param Ty:
    :param h:
    :param w:
    :return: model
    """
    input = Input(shape=(Tx, h, w))
    X = Reshape((Tx, h, w, 1))(input)
    X = TimeDistributed(Conv2D(32, (3, 3), padding='same'))(X)
    X = TimeDistributed(Conv2D(64, kernel_size=(3, 3), padding='same'))(X)
    X = TimeDistributed(Conv2D(1, kernel_size=(3, 3), padding='same'))(X)

    Y = ConvLSTM2D(filters=16, kernel_size=(3, 3), padding='same', return_sequences=True)(X)
    Y = ConvLSTM2D(filters=1, kernel_size=(3, 3), padding='same')(Y)

    output = Reshape((h, w))(Y)

    model = Model(input, output)

    return model


model = getModel(Tx, 40, 200)

print(model.summary())



model.compile(optimizer='RMSprop', loss='mean_squared_error')

model.fit(train_x, train_y, batch_size=100, epochs=10, verbose=1)